In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [79]:

import sys
sys.path.append('./')

import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np

import pickle
import faiss                     # make faiss available
import faiss.contrib.torch_utils
from tqdm import tqdm

from softgroup.pointnet2.pointnet2_utils import furthest_point_sample
from softgroup.model.geodesic_utils import cal_geodesic_vectorize2

import os


import faiss                     # make faiss available
import faiss.contrib.torch_utils

In [52]:
faiss_cfg = faiss.GpuIndexFlatConfig()
faiss_cfg.useFloat16 = True
faiss_cfg.device = 0

geo_knn = faiss.GpuIndexFlatL2(faiss.StandardGpuResources(), 3, faiss_cfg)

In [32]:
with open('/home/ubuntu/fewshot3d_ws/SoftGroup/results/bbox_context/sem_info/info_scene0011_00.pkl', 'rb') as handle:
    scene_info = pickle.load(handle)

In [33]:
coords_float = torch.from_numpy(scene_info['coords_float']).cuda()
semantic_scores = torch.from_numpy(scene_info['semantic_scores']).cuda()
semantic_scores = F.softmax(semantic_scores, dim=-1)

box_conf = torch.from_numpy(scene_info['box_conf']).cuda()


In [34]:
semantic_cls_preds = torch.argmax(semantic_scores, dim=-1)

object_idxs = torch.nonzero((semantic_cls_preds >= 2)).view(-1)

coords_float_ = coords_float[object_idxs]
semantic_scores_ = semantic_scores[object_idxs]
box_conf_ = box_conf[object_idxs]

In [80]:
conf_idxs = torch.nonzero(box_conf_ >= 0.6).view(-1)

# print(conf_idxs.shape)

candidates_float = coords_float_[conf_idxs].unsqueeze(0)
queries_inds = furthest_point_sample(candidates_float, 64).squeeze(0)

queries_idxs = conf_idxs[queries_inds.long()]

geo_dist = cal_geodesic_vectorize2(geo_knn, queries_idxs, coords_float_, semantic_scores_, n_queries=64)

feats_sim torch.Size([128900, 63]) tensor(-9.3516, device='cuda:0', dtype=torch.float16) tensor(-9.3281, device='cuda:0', dtype=torch.float16)


In [76]:
sum_len = torch.sum(geo_dist != -1, dim=-1)
print(torch.unique(sum_len))
print()

tensor([1], device='cuda:0')



In [50]:

save_dict = dict(
    object_idxs=object_idxs,
    queries_idxs=queries_idxs
)

with open(f'/home/ubuntu/fewshot3d_ws/SoftGroup/results/bbox_context/inds/inds_scene0011_00.pkl', 'wb') as handle:
    pickle.dump(save_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)